In [1]:
import pandas as pd 
import numpy as np 
import helper_functions.helpers as helpers
import h5py



In [2]:

def print_proportions(df):
    """
    # Print the total proportions!

    input: pandas df
    
    """
    rows, vars = df.shape
    print('')
    for l, vals in df.iteritems():
        #if str(l[1]) == 'mean':
        ratio = 1.0 * vals.dropna().count() / rows
        print(str(l) + ': ' + str(round(ratio, 3)*100) + '% present')

    return None 



def remove_dup_columns(frame):
     keep_names = set()
     keep_icols = list()
     for icol, name in enumerate(frame.columns):
          if name not in keep_names:
               keep_names.add(name)
               keep_icols.append(icol)
     return frame.iloc[:, keep_icols]

In [7]:
with h5py.File('D:\\Projects\\MIMIC\\Representation-Clustering\\patient_clustering_EHR\\data\\data_icu.h5') as f: 
    print(f.keys())


<KeysViewHDF5 ['antibiotics', 'colloid_bolus', 'crystalloid_bolus', 'demographics', 'diagnoses', 'labs_vitals', 'mech_vent', 'vaso_cv', 'vaso_mv']>


In [3]:
with h5py.File("C:\\Users\\\Maria\\Desktop\\Work\\Projects\\MIMIC\\Representation-Clustering\\patient_clustering_EHR\\data\\data_icu.h5") as f:
    print(f.keys())  # works like a dict

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'C:\Users\\Maria\Desktop\Work\Projects\MIMIC\Representation-Clustering\patient_clustering_EHR\data\data_icu.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [3]:
demographics = pd.read_hdf('C:\\Users\\\Maria\\Desktop\\Work\\Projects\\MIMIC\\Representation-Clustering\\patient_clustering_EHR\\data_latest\\demographics.h5', key='demographics') 
subject_id_set = set(demographics['subject_id'])
hadm_id_set = set(demographics['hadm_id'])
icustay_id_set = set(demographics['icustay_id'])


In [4]:
#NPUTEVENTS_CV was not used as it contains information regarding patients in the CareVue system,
#so it is not for the cohorts of interest. 
vaso_cv = pd.read_hdf('C:\\Users\\\Maria\\Desktop\\Work\\Projects\\MIMIC\\Representation-Clustering\\patient_clustering_EHR\\data\\data_icu.h5', key='vaso_cv')
vaso_cv

vaso_mv = pd.read_hdf('C:\\Users\\\Maria\\Desktop\\Work\\Projects\\MIMIC\\Representation-Clustering\\patient_clustering_EHR\\data\\data_icu.h5', key='vaso_mv')
vaso_mv

,subject_id,hadm_id,icustay_id,itemid,starttime,endtime,amount,amount,rate_std,has_vasopressors
0,41710,181955,200028,221906,2133-10-29 17:49:00,2133-10-29 18:11:00,0.110901,NaN,0.060,1
1,41710,181955,200028,221906,2133-10-29 18:11:00,2133-10-29 18:19:00,0.053759,NaN,0.080,1
2,41710,181955,200028,221906,2133-10-29 18:19:00,2133-10-29 18:28:00,0.075582,NaN,0.100,1
3,41710,181955,200028,221906,2133-10-29 18:28:00,2133-10-29 18:41:00,0.152844,NaN,0.140,1
4,41710,181955,200028,221906,2133-10-29 18:41:00,2133-10-29 18:49:00,0.107518,NaN,0.160,1
...,...,...,...,...,...,...,...,...,...,...
61349,42728,142693,299979,221749,2127-12-05 22:11:00,2127-12-05 22:42:00,1.606187,NaN,0.270,1
61350,42728,142693,299979,221749,2127-12-05 22:42:00,2127-12-06 00:42:00,4.142000,NaN,0.180,1
61351,42728,142693,299979,221749,2127-12-06 00:42:00,2127-12-06 01:22:00,1.381666,NaN,0.180,1
61352,42728,142693,299979,221749,2127-12-06 01:22:00,2127-12-06 03:20:00,3.056545,NaN,0.135,1


In [5]:
# sanity check, all patients are in the first ICU stay?
print(vaso_mv[vaso_mv.icustay_id.isin(demographics.icustay_id)])


vaso_mv = remove_dup_columns(vaso_mv)
vaso_mv.sort_values(['subject_id', 'starttime', 'endtime'], inplace=True)

#vaso_mv['started_after_hours'] = (vaso_mv['starttime'] - demographics['intime'])
# join and add in labs_vital the icu intime and outime, to separate in hourly buckets
demographics = demographics.set_index("icustay_id")
vaso_mv = vaso_mv.set_index('icustay_id').join(demographics[['intime', 'outtime']])

vaso_mv.sort_values(['subject_id', 'starttime', 'endtime'], inplace=True)


print(vaso_mv.itemid.value_counts())


vaso_mv = vaso_mv.replace({221749:'phenyl', 221906: 'phenyl', 221662:'dopa', 30047:"dopa", 221289:"epi", 222315:"vasopressin"})


print(vaso_mv.itemid.value_counts())

vaso_mv['started_after_hours'] = (vaso_mv['starttime'] - vaso_mv['intime'])

vaso_mv['vaso_lasted'] = (vaso_mv['endtime'] - vaso_mv['starttime'])

to_hours = lambda x: max(0, x.days*24 + x.seconds // 3600)
vaso_mv['hours_in'] = (vaso_mv['starttime'] - vaso_mv['intime']).apply(to_hours)

vaso = vaso_mv.drop(columns=['starttime', 'endtime', 'rate_std', 'has_vasopressors', 'intime', 'outtime', 'started_after_hours', 'vaso_lasted'])

vaso = vaso.reset_index()

print("Aggregate and get stats")
#labs_vitals_stats = labs_vitals.groupby(['subject_id', 'hadm_id', 'icustay_id'] + ['LEVEL2'] + ['hours_in']).agg(['mean', 'std', 'count'])
vaso = vaso.groupby(['subject_id', 'hadm_id', 'icustay_id'] + ['hours_in'] + ['itemid']).agg(['mean'])

#vaso.drop('amount', axis=1, level=0, inplace=True)
vaso.columns = vaso.columns.droplevel(1)
#vaso.columns.names = ['Aggregated']

demographics['max_hours'] = (demographics['outtime'] - demographics['intime']).apply(to_hours)
# Pivot table droups NaN columns so you lose any uniformly NaN.


vaso = vaso.unstack(level=4)


print_proportions(vaso)

       subject_id  hadm_id  icustay_id  itemid           starttime  \
0           41710   181955      200028  221906 2133-10-29 17:49:00   
1           41710   181955      200028  221906 2133-10-29 18:11:00   
2           41710   181955      200028  221906 2133-10-29 18:19:00   
3           41710   181955      200028  221906 2133-10-29 18:28:00   
4           41710   181955      200028  221906 2133-10-29 18:41:00   
...           ...      ...         ...     ...                 ...   
61349       42728   142693      299979  221749 2127-12-05 22:11:00   
61350       42728   142693      299979  221749 2127-12-05 22:42:00   
61351       42728   142693      299979  221749 2127-12-06 00:42:00   
61352       42728   142693      299979  221749 2127-12-06 01:22:00   
61353       42728   142693      299979  221749 2127-12-06 03:20:00   

                  endtime     amount amount  rate_std  has_vasopressors  
0     2133-10-29 18:11:00   0.110901    NaN     0.060                 1  
1     2133-

c:\Users\Maria\Desktop\Work\Projects\MIMIC\Representation-Clustering\.venv\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


phenyl         54006
dopa            3814
epi             2309
vasopressin     1225
Name: itemid, dtype: int64
Aggregate and get stats

('amount', 'dopa'): 6.9% present
('amount', 'epi'): 4.7% present
('amount', 'phenyl'): 90.2% present
('amount', 'vasopressin'): 2.7% present


In [6]:
## colloid bolus

In [7]:
colloid_b = pd.read_hdf('C:\\Users\\\Maria\\Desktop\\Work\\Projects\\MIMIC\\Representation-Clustering\\patient_clustering_EHR\\data\\data_icu.h5', key='colloid_bolus')
demographics.reset_index(inplace=True)
# sanity check, all patients are in the first ICU stay?

print(colloid_b[colloid_b.icustay_id.isin(demographics.icustay_id)])

demographics = demographics.set_index("icustay_id")
colloid_b = colloid_b.set_index('icustay_id').join(demographics[['intime', 'outtime']])

colloid_b.sort_values(['subject_id', 'charttime'], inplace=True)

colloid_b['hours_in'] = (colloid_b['charttime'] - colloid_b['intime']).apply(to_hours)

colloid_b = colloid_b.drop(columns=['charttime', 'intime', 'outtime'])

colloid_b = colloid_b.reset_index()
colloid_b.head()

       subject_id   hadm_id  icustay_id           charttime  colloid_bolus
0              20  157681.0      264490 2183-04-28 23:30:00          500.0
2             114  178393.0      258626 2146-08-30 05:00:00          500.0
3             114  178393.0      258626 2146-08-30 12:00:00          500.0
4             115  114585.0      232514 2194-10-17 16:00:00          500.0
5             115  114585.0      232514 2194-10-17 22:00:00          500.0
...           ...       ...         ...                 ...            ...
12174       99982  151454.0      221194 2156-11-29 21:35:00          500.0
12175       99982  151454.0      221194 2156-11-29 22:16:00          500.0
12176       99982  151454.0      221194 2156-11-29 22:56:00          500.0
12177       99982  151454.0      221194 2156-12-04 11:26:00          250.0
12178       99982  151454.0      221194 2156-12-05 17:50:00          250.0

[5654 rows x 5 columns]


,icustay_id,subject_id,hadm_id,colloid_bolus,hours_in
0,264490,20,157681.0,500.0,8
1,258626,114,178393.0,500.0,11
2,258626,114,178393.0,500.0,18
3,232514,115,114585.0,500.0,2
4,232514,115,114585.0,500.0,8


In [8]:
colloid_b.set_index(['icustay_id', 'subject_id', 'hadm_id', 'hours_in'], inplace=True)
colloid_b

colloid_bolus
icustay_id subject_id hadm_id  hours_in               
264490     20         157681.0 8                 500.0
258626     114        178393.0 11                500.0
                               18                500.0
232514     115        114585.0 2                 500.0
                               8                 500.0
...                                                ...
221194     99982      151454.0 33                500.0
                               34                500.0
                               34                500.0
                               143               250.0
                               173               250.0

[5654 rows x 1 columns]

In [9]:
colloid_b = colloid_b.groupby(['subject_id', 'hadm_id', 'icustay_id'] + ['hours_in'] ).agg(['mean'])

colloid_b.columns = colloid_b.columns.droplevel(1)

In [10]:
colloid_b

colloid_bolus
subject_id hadm_id  icustay_id hours_in               
20         157681.0 264490     8                 500.0
114        178393.0 258626     11                500.0
                               18                500.0
115        114585.0 232514     2                 500.0
                               8                 500.0
...                                                ...
99938      103815.0 296707     27                500.0
99982      151454.0 221194     33                500.0
                               34                500.0
                               143               250.0
                               173               250.0

[5481 rows x 1 columns]

In [11]:
## crystalloid b 

crystalloid_bolus = pd.read_hdf('C:\\Users\\\Maria\\Desktop\\Work\\Projects\\MIMIC\\Representation-Clustering\\patient_clustering_EHR\\data\\data_icu.h5', key='crystalloid_bolus')
demographics.reset_index(inplace=True)
# sanity check, all patients are in the first ICU stay?

print(crystalloid_bolus[crystalloid_bolus.icustay_id.isin(demographics.icustay_id)])

demographics = demographics.set_index("icustay_id")
crystalloid_bolus = crystalloid_bolus.set_index('icustay_id').join(demographics[['intime', 'outtime']])

crystalloid_bolus.sort_values(['subject_id', 'charttime'], inplace=True)

crystalloid_bolus['hours_in'] = (crystalloid_bolus['charttime'] - crystalloid_bolus['intime']).apply(to_hours)

crystalloid_bolus = crystalloid_bolus.drop(columns=['charttime', 'intime', 'outtime'])

crystalloid_bolus = crystalloid_bolus.reset_index()
crystalloid_bolus.head()

        subject_id   hadm_id  icustay_id           charttime  \
0                3  145834.0      211552 2101-10-21 11:00:00   
1                3  145834.0      211552 2101-10-21 14:00:00   
2                3  145834.0      211552 2101-10-21 17:30:00   
3                3  145834.0      211552 2101-10-21 20:00:00   
4                3  145834.0      211552 2101-10-21 22:30:00   
...            ...       ...         ...                 ...   
155070       99982  151454.0      221194 2156-11-29 12:59:00   
155071       99982  151454.0      221194 2156-11-29 13:22:00   
155072       99982  151454.0      221194 2156-11-29 15:57:00   
155073       99982  151454.0      221194 2156-11-29 16:25:00   
155085       99995  137810.0      229633 2147-02-08 17:00:00   

        crystalloid_bolus  
0                   500.0  
1                   820.0  
2                   500.0  
3                   500.0  
4                   500.0  
...                   ...  
155070             1000.0  
155071 

,icustay_id,subject_id,hadm_id,crystalloid_bolus,hours_in
0,211552,3,145834.0,500.0,15
1,211552,3,145834.0,820.0,18
2,211552,3,145834.0,500.0,22
3,211552,3,145834.0,500.0,24
4,211552,3,145834.0,500.0,27


In [12]:
crystalloid_bolus = crystalloid_bolus.groupby(['subject_id', 'hadm_id', 'icustay_id'] + ['hours_in'] ).agg(['mean'])

crystalloid_bolus.columns = crystalloid_bolus.columns.droplevel(1)

In [13]:
crystalloid_bolus

crystalloid_bolus
subject_id hadm_id  icustay_id hours_in                   
3          145834.0 211552     15                    500.0
                               18                    820.0
                               22                    500.0
                               24                    500.0
                               27                    500.0
...                                                    ...
99973      150202.0 275083     12                    500.0
99982      151454.0 221194     25                   1000.0
                               27                    250.0
                               28                   1000.0
99995      137810.0 229633     3                     500.0

[74784 rows x 1 columns]

In [14]:
##crystalloid_bolus.set_index(['icustay_id', 'subject_id', 'hadm_id', 'hours_in'], inplace=True)
#crystalloid_bolus

In [15]:
# joining bolus tables
bolus = colloid_b.join(crystalloid_bolus, how='outer')
bolus = bolus.sort_index()
bolus.head()

colloid_bolus  crystalloid_bolus
subject_id hadm_id  icustay_id hours_in                                  
3          145834.0 211552     15                  NaN              500.0
                               18                  NaN              820.0
                               22                  NaN              500.0
                               24                  NaN              500.0
                               27                  NaN              500.0

In [16]:
bolus.reset_index(inplace=True)

In [17]:
#sanity check for correct joining
bolus[bolus.subject_id == 93071]

,subject_id,hadm_id,icustay_id,hours_in,colloid_bolus,crystalloid_bolus
76273,93071,197473.0,200141,3,NaN,1000.0
76274,93071,197473.0,200141,4,500.0,1000.0
76275,93071,197473.0,200141,11,500.0,NaN


In [18]:
colloid_b.reset_index(inplace=True)
crystalloid_bolus.reset_index(inplace=True)


In [19]:
colloid_b[colloid_b.subject_id == 93071]


,subject_id,hadm_id,icustay_id,hours_in,colloid_bolus
5061,93071,197473.0,200141,4,500.0
5062,93071,197473.0,200141,11,500.0


In [20]:
crystalloid_bolus[crystalloid_bolus.subject_id == 93071]

,subject_id,hadm_id,icustay_id,hours_in,crystalloid_bolus
71670,93071,197473.0,200141,3,1000.0
71671,93071,197473.0,200141,4,1000.0


In [21]:
#join vaso and bolus
vaso.index

MultiIndex([(  877, 166453, 209705,   5),
            (  877, 166453, 209705,   7),
            (  877, 166453, 209705,  10),
            (  877, 166453, 209705,  11),
            (  877, 166453, 209705,  12),
            (  877, 166453, 209705,  14),
            (  877, 166453, 209705,  17),
            ( 4974, 115247, 204366,   3),
            ( 5685, 193803, 208236,   1),
            ( 5685, 193803, 208236,   2),
            ...
            (99982, 151454, 221194,  66),
            (99982, 151454, 221194,  68),
            (99982, 151454, 221194,  72),
            (99982, 151454, 221194,  76),
            (99982, 151454, 221194,  78),
            (99982, 151454, 221194,  82),
            (99982, 151454, 221194,  89),
            (99982, 151454, 221194, 107),
            (99982, 151454, 221194, 117),
            (99995, 137810, 229633,   1)],
           names=['subject_id', 'hadm_id', 'icustay_id', 'hours_in'], length=42141)

In [22]:
bolus.set_index(['icustay_id', 'subject_id', 'hadm_id', 'hours_in'], inplace=True)
bolus.head()

colloid_bolus  crystalloid_bolus
icustay_id subject_id hadm_id  hours_in                                  
211552     3          145834.0 15                  NaN              500.0
                               18                  NaN              820.0
                               22                  NaN              500.0
                               24                  NaN              500.0
                               27                  NaN              500.0

In [23]:
vaso_bolus = bolus.join(vaso, how='outer')
vaso_bolus.head()

c:\Users\Maria\Desktop\Work\Projects\MIMIC\Representation-Clustering\.venv\lib\site-packages\pandas\core\frame.py:9138: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  sort=sort,


colloid_bolus  crystalloid_bolus  \
icustay_id subject_id hadm_id  hours_in                                     
200003     27513      163557.0 1                   NaN             1000.0   
                               2                   NaN             1000.0   
                               3                   NaN             1000.0   
                               5                 250.0                NaN   
200007     20707      129310.0 9                   NaN              320.0   

                                         (amount, dopa)  (amount, epi)  \
icustay_id subject_id hadm_id  hours_in                                  
200003     27513      163557.0 1                    NaN            NaN   
                               2                    NaN            NaN   
                               3                    NaN            NaN   
                               5                    NaN            NaN   
200007     20707      129310.0 9                    NaN            NaN   

                                         (amount, phenyl)  \
icustay_id subject_id hadm_id  hours_in                     
200003     27513      163557.0 1                      NaN   
                               2                      NaN   
                               3                      NaN   
                               5                      NaN   
200007     20707      129310.0 9                      NaN   

                                         (amount, vasopressin)  
icustay_id subject_id hadm_id  hours_in                         
200003     27513      163557.0 1                           NaN  
                               2                           NaN  
                               3                           NaN  
                               5                           NaN  
200007     20707      129310.0 9                           NaN

In [24]:
# sanity check 
vaso_bolus.reset_index(inplace=True)

In [25]:
vaso_bolus[vaso_bolus.subject_id == 93071]

,icustay_id,subject_id,hadm_id,hours_in,colloid_bolus,crystalloid_bolus,"(amount, dopa)","(amount, epi)","(amount, phenyl)","(amount, vasopressin)"
119,200141,93071,197473.0,2,NaN,NaN,NaN,NaN,0.735502,NaN
120,200141,93071,197473.0,3,NaN,1000.0,NaN,NaN,0.546079,NaN
121,200141,93071,197473.0,4,500.0,1000.0,NaN,NaN,0.337146,NaN
122,200141,93071,197473.0,11,500.0,NaN,NaN,NaN,NaN,NaN


In [26]:
vaso.reset_index(inplace=True)
vaso[vaso.subject_id == 93071]

subject_id hadm_id icustay_id hours_in amount                          
itemid                                          dopa epi    phenyl vasopressin
37334       93071  197473     200141        2    NaN NaN  0.735502         NaN
37335       93071  197473     200141        3    NaN NaN  0.546079         NaN
37336       93071  197473     200141        4    NaN NaN  0.337146         NaN

In [27]:
vaso_bolus

,icustay_id,subject_id,hadm_id,hours_in,colloid_bolus,crystalloid_bolus,"(amount, dopa)","(amount, epi)","(amount, phenyl)","(amount, vasopressin)"
0,200003,27513,163557.0,1,NaN,1000.0,NaN,NaN,NaN,NaN
1,200003,27513,163557.0,2,NaN,1000.0,NaN,NaN,NaN,NaN
2,200003,27513,163557.0,3,NaN,1000.0,NaN,NaN,NaN,NaN
3,200003,27513,163557.0,5,250.0,NaN,NaN,NaN,NaN,NaN
4,200007,20707,129310.0,9,NaN,320.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
114851,299998,69587,158288.0,38,NaN,1000.0,NaN,NaN,NaN,NaN
114852,299999,7630,129161.0,12,NaN,450.0,NaN,NaN,NaN,NaN
114853,299999,7630,129161.0,14,NaN,300.0,NaN,NaN,NaN,NaN
114854,299999,7630,129161.0,17,NaN,500.0,NaN,NaN,NaN,NaN


In [28]:
vaso_bolus = vaso_bolus.rename(columns={('amount', 'dopa'): "dopa", ('amount', 'phenyl'): "phenyl", ('amount', 'epi'):"epi", ('amount', 'vasopressin'):"vasopressin"})

In [29]:
vaso_bolus.head()

,icustay_id,subject_id,hadm_id,hours_in,colloid_bolus,crystalloid_bolus,dopa,epi,phenyl,vasopressin
0,200003,27513,163557.0,1,NaN,1000.0,NaN,NaN,NaN,NaN
1,200003,27513,163557.0,2,NaN,1000.0,NaN,NaN,NaN,NaN
2,200003,27513,163557.0,3,NaN,1000.0,NaN,NaN,NaN,NaN
3,200003,27513,163557.0,5,250.0,NaN,NaN,NaN,NaN,NaN
4,200007,20707,129310.0,9,NaN,320.0,NaN,NaN,NaN,NaN


In [30]:
vaso_bolus.to_hdf('interventions.h5', key='vasobolus_before48') 

In [31]:
vaso_bolus

,icustay_id,subject_id,hadm_id,hours_in,colloid_bolus,crystalloid_bolus,dopa,epi,phenyl,vasopressin
0,200003,27513,163557.0,1,NaN,1000.0,NaN,NaN,NaN,NaN
1,200003,27513,163557.0,2,NaN,1000.0,NaN,NaN,NaN,NaN
2,200003,27513,163557.0,3,NaN,1000.0,NaN,NaN,NaN,NaN
3,200003,27513,163557.0,5,250.0,NaN,NaN,NaN,NaN,NaN
4,200007,20707,129310.0,9,NaN,320.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
114851,299998,69587,158288.0,38,NaN,1000.0,NaN,NaN,NaN,NaN
114852,299999,7630,129161.0,12,NaN,450.0,NaN,NaN,NaN,NaN
114853,299999,7630,129161.0,14,NaN,300.0,NaN,NaN,NaN,NaN
114854,299999,7630,129161.0,17,NaN,500.0,NaN,NaN,NaN,NaN


In [32]:
# keep 48h


#keep 48h
print("keeping 48h....")
vasobolus_reset = vaso_bolus.copy()

#del vaso_bolus
df_to_keep = []
patients_less_48 = 0
patients_more_48 = 0
for i, icu, hadm in zip(set(vasobolus_reset.subject_id), set(vasobolus_reset.icustay_id), set(vasobolus_reset.hadm_id)):
    print(i)
    max_icu_patient_time = vasobolus_reset[vasobolus_reset.subject_id == i]['hours_in'].max()
    #[labs_vitals_reset.subject_id == i[0]]
    min_icu_patient_time = vasobolus_reset[vasobolus_reset.subject_id == i]['hours_in'].max() - 48
    #print(max_icu_patient_time, min_icu_patient_time)
    if max_icu_patient_time >= 48:

        patients_more_48 = patients_more_48 + 1
        print(">48h", i)
        temp = vasobolus_reset[vasobolus_reset.subject_id == i]
        temp = temp[(temp['hours_in'] >=min_icu_patient_time) & (temp['hours_in'] <=max_icu_patient_time)]
        
        #temp = temp.set_index('hours_in').reindex(range(temp['hours_in'].min(), temp['hours_in'].max())).reset_index()
        temp = temp.set_index('hours_in').reindex(range(min_icu_patient_time, max_icu_patient_time)).reset_index()
        print(range(min_icu_patient_time, max_icu_patient_time))
        
        subject_id = [i] * len(set(temp.index))
        icustay_id = [icu] * len(set(temp.index))
        hadm_id = [hadm] * len(set(temp.index))

        hours_in = list(range(len(set(temp.index))))
        temp['hours_in_'] = hours_in
        temp['subject_id'] = subject_id
        temp['icustay_id'] = icustay_id
        temp['hadm_id'] = hadm_id

        df_to_keep.append(temp)
    else:
        patients_less_48 = patients_less_48 + 1
        print("<48h", i)

print("len of patient with more than 48", patients_more_48 )
print("len of patient with less than 48", patients_less_48 )

keeping 48h....
3
<48h 3
4
<48h 4
17
<48h 17
20
<48h 20
22
<48h 22
23
<48h 23
25
<48h 25
28
<48h 28
30
<48h 30
31
>48h 31
range(109, 157)
32
<48h 32
35
>48h 35
range(29, 77)
36
<48h 36
43
<48h 43
44
<48h 44
45
<48h 45
49
<48h 49
55
<48h 55
61
<48h 61
62
<48h 62
64
>48h 64
range(6, 54)
68
<48h 68
71
>48h 71
range(2, 50)
73
<48h 73
75
<48h 75
77
<48h 77
80
<48h 80
84
<48h 84
85
<48h 85
86
<48h 86
94
<48h 94
98
>48h 98
range(1, 49)
99
<48h 99
101
>48h 101
range(67, 115)
103
>48h 103
range(16, 64)
105
>48h 105
range(70, 118)
106
>48h 106
range(64, 112)
108
<48h 108
111
>48h 111
range(58, 106)
114
<48h 114
115
<48h 115
119
<48h 119
123
<48h 123
130
<48h 130
133
<48h 133
134
<48h 134
135
<48h 135
136
<48h 136
137
<48h 137
138
<48h 138
142
<48h 142
144
<48h 144
145
>48h 145
range(12, 60)
147
>48h 147
range(41, 89)
149
<48h 149
151
>48h 151
range(57, 105)
152
<48h 152
154
<48h 154
155
<48h 155
158
<48h 158
160
<48h 160
169
<48h 169
171
<48h 171
173
<48h 173
174
>48h 174
range(44, 92)
176
<48h 

In [33]:
vaso_bolus_48 = pd.concat(df_to_keep)
del df_to_keep
vaso_bolus_48.head()

,hours_in,icustay_id,subject_id,hadm_id,colloid_bolus,crystalloid_bolus,dopa,epi,phenyl,vasopressin,hours_in_
0,109,262226,31,131133.0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,110,262226,31,131133.0,NaN,NaN,NaN,NaN,NaN,NaN,1
2,111,262226,31,131133.0,NaN,NaN,NaN,NaN,NaN,NaN,2
3,112,262226,31,131133.0,NaN,NaN,NaN,NaN,NaN,NaN,3
4,113,262226,31,131133.0,NaN,NaN,NaN,NaN,NaN,NaN,4


In [34]:
import numpy as np 
# Do linear interpolation where missingness is low (kNN imputation doesnt work if all rows have missing values)
print('Full ICU -- Doing linear interpolation where missingness is low (kNN imputation doesnt work if all rows have missing values)')
miss = np.sum(np.isnan(vaso_bolus_48), axis=0)/vaso_bolus_48.shape[0]
#ii = (miss>0) & (miss<0.05)  #less than 5% missingness
#mechventcol = reformat3t.columns.tolist().index('mechvent')
print(miss)


Full ICU -- Doing linear interpolation where missingness is low (kNN imputation doesnt work if all rows have missing values)
hours_in             0.000000
icustay_id           0.000000
subject_id           0.000000
hadm_id              0.000000
colloid_bolus        0.995545
crystalloid_bolus    0.943455
dopa                 0.996350
epi                  0.998275
phenyl               0.947994
vasopressin          0.998100
hours_in_            0.000000
dtype: float64


In [38]:
# missing values - linear interpolation
vaso_bolus_48.set_index(['subject_id', 'hadm_id', 'icustay_id'], inplace=True)
vaso_bolus_48 = vaso_bolus_48.sort_index(axis=0).sort_index(axis=1)
vaso_bolus_48.head()

colloid_bolus  crystalloid_bolus  dopa  epi  \
subject_id hadm_id  icustay_id                                                
31         131133.0 262226                NaN                NaN   NaN  NaN   
                    262226                NaN                NaN   NaN  NaN   
                    262226                NaN                NaN   NaN  NaN   
                    262226                NaN                NaN   NaN  NaN   
                    262226                NaN                NaN   NaN  NaN   

                                hours_in  hours_in_  phenyl  vasopressin  
subject_id hadm_id  icustay_id                                            
31         131133.0 262226           109          0     NaN          NaN  
                    262226           110          1     NaN          NaN  
                    262226           111          2     NaN          NaN  
                    262226           112          3     NaN          NaN  
                    262226           113          4     NaN          NaN

In [39]:
########################################################################
#             HANDLING OF MISSING VALUES  &  CREATE REFORMAT4T
########################################################################
#df.interpolate(limit=1, limit_direction="forward");

#imputer = KNNImputer(n_neighbors=1, weights='uniform', metric='nan_euclidean')

imputed_to_keep = []
for i in range(0,  vaso_bolus_48.shape[0], 48):
    temp = vaso_bolus_48.loc[[vaso_bolus_48.index[i][0]]]
    for col, val in temp.iteritems():
        #print(col)
        if (col != 'hours_in') and (col != 'hours_in_') and (~np.all(temp[col].isna())):
            #(str(col[1]) == 'mean') and 
            print('patient', vaso_bolus_48.index[i][0], 'feature', col)
            t = np.array(temp[col]).reshape(-1, 1)

            #imputed_col = imputer.fit_transform(t)
            imputed_col = temp[col].interpolate()
            temp.drop(col, axis=1, inplace=True)
            temp[col] = imputed_col
    imputed_to_keep.append(temp)

vital_labs_48_imp = pd.concat(imputed_to_keep)

del imputed_to_keep
vital_labs_48_imp = vital_labs_48_imp.fillna(0)

vital_labs_48_imp.to_hdf('interventions.h5', key='preprocessed') 

patient 31 feature crystalloid_bolus
patient 64 feature crystalloid_bolus
patient 71 feature crystalloid_bolus
patient 101 feature crystalloid_bolus
patient 106 feature crystalloid_bolus
patient 145 feature crystalloid_bolus
patient 151 feature colloid_bolus
patient 151 feature crystalloid_bolus
patient 174 feature crystalloid_bolus
patient 178 feature crystalloid_bolus
patient 191 feature crystalloid_bolus
patient 225 feature crystalloid_bolus
patient 245 feature colloid_bolus
patient 245 feature crystalloid_bolus
patient 252 feature crystalloid_bolus
patient 281 feature crystalloid_bolus
patient 314 feature crystalloid_bolus
patient 330 feature crystalloid_bolus
patient 344 feature crystalloid_bolus
patient 367 feature crystalloid_bolus
patient 370 feature crystalloid_bolus
patient 376 feature crystalloid_bolus
patient 394 feature crystalloid_bolus
patient 454 feature crystalloid_bolus
patient 507 feature crystalloid_bolus
patient 538 feature crystalloid_bolus
patient 540 feature cry

In [40]:
vital_labs_48_imp

colloid_bolus  dopa  epi  hours_in  hours_in_  \
subject_id hadm_id  icustay_id                                                  
31         131133.0 262226                0.0   0.0  0.0       109          0   
                    262226                0.0   0.0  0.0       110          1   
                    262226                0.0   0.0  0.0       111          2   
                    262226                0.0   0.0  0.0       112          3   
                    262226                0.0   0.0  0.0       113          4   
...                                       ...   ...  ...       ...        ...   
99982      131014.0 262140              250.0   0.0  0.0       168         43   
                    262140              250.0   0.0  0.0       169         44   
                    262140              250.0   0.0  0.0       170         45   
                    262140              250.0   0.0  0.0       171         46   
                    262140              250.0   0.0  0.0       172         47   

                                phenyl  vasopressin  crystalloid_bolus  
subject_id hadm_id  icustay_id                                          
31         131133.0 262226         0.0          0.0                0.0  
                    262226         0.0          0.0                0.0  
                    262226         0.0          0.0                0.0  
                    262226         0.0          0.0                0.0  
                    262226         0.0          0.0                0.0  
...                                ...          ...                ...  
99982      131014.0 262140         0.0          0.0                0.0  
                    262140         0.0          0.0                0.0  
                    262140         0.0          0.0                0.0  
                    262140         0.0          0.0                0.0  
                    262140         0.0          0.0                0.0  

[200016 rows x 8 columns]